In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch
import torch.nn.functional as F
from transformers import BertModel, BertTokenizerFast
from torch import nn
import numpy as np
import pandas as pd


In [3]:
df_extra = pd.read_csv('/content/descriptions.csv')

In [4]:
#This notebook is ran in Google Colab to use GPU for LaBSE embeddings
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [5]:
tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
model = BertModel.from_pretrained("setu4993/LaBSE")
model.cuda()
model = model.eval()

In [6]:
def similarity(embeddings_1, embeddings_2):
    normalized_embeddings_1 = F.normalize(embeddings_1, p=2)
    normalized_embeddings_2 = F.normalize(embeddings_2, p=2)
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    
    return cos(embeddings_1, embeddings_2)

In [7]:
descriptions = list(df_extra['Description'])

In [8]:
lista_tensors = []
ln = 0
rn = 200
for i in range(26):
  print(ln,rn)
  torch.cuda.empty_cache()
  if rn < 4672:
    descriptions_1 = tokenizer(descriptions[ln:rn], return_tensors="pt", padding=True, truncation=True,max_length=512)
    descriptions_1.to(device)
    with torch.no_grad():
      descriptions_outputs = model(**descriptions_1)
      lista_tensors.append(descriptions_outputs.pooler_output)
  if rn > 4672 and ln <= 4672 :
    descriptions_1 = tokenizer(descriptions[ln:4672], return_tensors="pt", padding=True, truncation=True,max_length=512)
    descriptions_1.to(device)
    with torch.no_grad():
      descriptions_outputs = model(**descriptions_1)
      lista_tensors.append(descriptions_outputs.pooler_output)
  
  ln += 200
  rn += 200
  

0 200
200 400
400 600
600 800
800 1000
1000 1200
1200 1400
1400 1600
1600 1800
1800 2000
2000 2200
2200 2400
2400 2600
2600 2800
2800 3000
3000 3200
3200 3400
3400 3600
3600 3800
3800 4000
4000 4200
4200 4400
4400 4600
4600 4800
4800 5000
5000 5200


In [28]:
final_tensor = torch.Tensor()
for tensors in lista_tensors[:]:
  final_tensor = torch.cat((final_tensor.to(device), tensors.to(device)), 0)


In [36]:
final_tensor = final_tensor.cpu().detach().numpy()

In [38]:
df_extra['LaBSE_emb'] = final_tensor.tolist()

In [45]:
df_extra.to_csv('df_extra_emb.csv',index=True)